Prepare environment

In [ ]:
import pandas as pd
import re
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

nltk.download("stopwords")
nltk.download("wordnet")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\appul\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\appul\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\appul\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

Load the files

In [5]:
#Load the training data
train=pd.read_csv("data/training_data_lowercase.csv",
                   sep="\t",
                   header=None,
                   names=["label","text"],
                   engine="python")

#Load the testing data
test=pd.read_csv("data/testing_data_lowercase_nolabels.csv",
                  header=None,
                  names=["text"])

print("Train shape:", train.shape)
print("Test shape:", test.shape)
print(train.head())
print(train.columns.tolist())

Train shape: (68307, 2)
Test shape: (19971, 1)
          label                                               text
0  <<<<<<< HEAD                                               None
1            ﻿0  donald trump sends out embarrassing new year‚s...
2             0  drunk bragging trump staffer started russian c...
3             0  sheriff david clarke becomes an internet joke ...
4             0  trump is so obsessed he even has obama‚s name ...
['label', 'text']


Preprocessing

In [ ]:
def clean_text(text):
    #Remove urls and html
    text.re.sub(r"http\S+|www\S+|https\S+", " ", text)
    text=re.sub(r"<.*?>","", text)
    #Only alphabets and space
    text=re

Split, vectorizing and naive bayes

In [144]:
X=train_df["clean_text"]
y=train_df["label"]

X_train,X_val,y_train,y_val=train_test_split(
    X,y,test_size=0.2, random_state=42,stratify=y
)

vectorizer=TfidfVectorizer(
    stop_words="english",
    ngram_range=(1,2),
    max_features=20000,
    min_df=2
)

X_train_tfidf=vectorizer.fit_transform(X_train)
X_val_tfidf=vectorizer.transform(X_val)
X_test_tfidf=vectorizer.transform(test_df["clean_text"])

nb=MultinomialNB(alpha=1.0)
nb.fit(X_train_tfidf,y_train)

y_val_pred=nb.predict(X_val_tfidf)
acc=accuracy_score(y_val,y_val_pred)

print(f"\n Validation Accuracy:{acc:4f}\n")
print("classification_report:\n", classification_report(y_val,y_val_pred, digits=4))
print("Confusion Matrix:\n", confusion_matrix(y_val,y_pred))

KeyError: 'label'

Feature Extraction TF-IDF

In [ ]:
#Initialize the TfidfVectorizer
tfidf_vectorizer=TfidfVectorizer()

#Fit and transform the corpus into a TF-IDF representation
X_tfidf=tfidf_vectorizer.fit_transform(corpus)

#Show results
print("TF-IDF:\n", X_tfidf.toarray())
print("Vocabulary:", tfidf_vectorizer.get_feature_names_out())

Vectorize with Naive Bayes

In [ ]:
def text_preprocessing_pipeline(text):
    # Step 1: Tokenize the text
    tokens= word_tokenize(text.lower())
    # Step 2: Remove stop words
    stop_words=set(stopwords.words('english'))
    tokens=[token for token in tokens if token not in stop_words]
    # Step 3: Remove punctuation
    tokens=[token for token in tokens if token not in string.punctuation]
    # Step 4: Apply lemmatization
    lemmatizer=WordNetLemmatizer()
    lemmatized_tokens=[lemmatizer.lemmatize(token) for token in tokens]

    return lemmatized_tokens

In [ ]:
model=make_pipeline(
    TfidfVectorizer(
        lowercase=True,
        stop_words="english",
        ngram_range=(1,2), #try 1-3
        min_df=2,
        max_df=0.9,
        max_features=50000
    ),
    MultinomialNB(alpha=0.5) #try 0.1 and 1.0
)

model.fit(X_train,y_train)

NameError: name 'X_train' is not defined

Evaluate on validation set

In [ ]:
y_pred=model.predict(X_val)
print("Validation accuracy:", round(accuracy_score(y_val,y_pred),4))
print(classification_report(y_val,y_pred,digits=3))
print(confusion_matrix(y_val,y_pred))


Now train on all data and predict test

In [ ]:
#retrain on full training data
model.fit(train_df["text"], train_df["label"])

#Predict labels for test
test_pred=model.predict(test_df["text"]).astype(int)

